# Text generation with a miniature GPT

**Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)<br>
**Date created:** 2020/05/29<br>
**Last modified:** 2020/05/29<br>
**Description:** Implement a miniature version of GPT and train it to generate text.

## Introduction

This example demonstrates how to implement an autoregressive language model
using a miniature version of the GPT model.
The model consists of a single Transformer block with causal masking
in its attention layer.
We use the text from the IMDB sentiment classification dataset for training
and generate new movie reviews for a given prompt.
When using this script with your own dataset, make sure it has at least
1 million words.

This example should be run with `tf-nightly>=2.3.0-dev20200531` or
with TensorFlow 2.3 or higher.

**References:**

- [GPT](https://www.semanticscholar.org/paper/Improving-Language-Understanding-by-Generative-Radford/cd18800a0fe0b668a1cc19f2ec95b5003d0a5035)
- [GPT-2](https://www.semanticscholar.org/paper/Language-Models-are-Unsupervised-Multitask-Learners-Radford-Wu/9405cc0d6169988371b2755e573cc28650d14dfe)
- [GPT-3](https://arxiv.org/abs/2005.14165)

## Setup

In [1]:
# We set the backend to TensorFlow. The code works with
# both `tensorflow` and `torch`. It does not work with JAX
# due to the behavior of `jax.numpy.tile` in a jit scope
# (used in `causal_attention_mask()`: `tile` in JAX does
# not support a dynamic `reps` argument.
# You can make the code work in JAX by wrapping the
# inside of the `causal_attention_mask` function in
# a decorator to prevent jit compilation:
# `with jax.ensure_compile_time_eval():`.
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization
import numpy as np
import os
import string
import random
import tensorflow
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings


## Implement a Transformer block as a layer

In [2]:

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = ops.arange(n_dest)[:, None]
    j = ops.arange(n_src)
    m = i >= j - n_src + n_dest
    mask = ops.cast(m, dtype)
    mask = ops.reshape(mask, [1, n_dest, n_src])
    mult = ops.concatenate(
        [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])], 0
    )
    return ops.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, "bool")
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)


## Implement an embedding layer

Create two separate embedding layers: one for tokens and one for token index
(positions).

In [3]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(0, maxlen, 1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


## Implement the miniature GPT model

In [4]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 80  # Max sequence size
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs = layers.Input(shape=(maxlen,), dtype="int32")
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block(x)
    outputs = layers.Dense(vocab_size)(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam",
        loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model


## Prepare the data for word-level language modelling

Download the IMDB dataset and combine training and validation sets for a text
generation task.

In [5]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  42.4M      0  0:00:01  0:00:01 --:--:-- 42.4M


In [6]:

batch_size = 128

# The dataset contains each review in a separate text file
# The text files are present in four different folders
# Create a list all files
filenames = []
directories = [
    "aclImdb/train/pos",
    "aclImdb/train/neg",
    "aclImdb/test/pos",
    "aclImdb/test/neg",
]
for dir in directories:
    for f in os.listdir(dir):
        filenames.append(os.path.join(dir, f))

print(f"{len(filenames)} files")

# Create a dataset from text files
random.shuffle(filenames)
text_ds = tf_data.TextLineDataset(filenames)
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)


def custom_standardization(input_string):
    """Remove html line-break tags and handle punctuation"""
    lowercased = tf_strings.lower(input_string)
    stripped_html = tf_strings.regex_replace(lowercased, "<br />", " ")
    return tf_strings.regex_replace(stripped_html, f"([{string.punctuation}])", r" \1")


# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()  # To get words back from token indices


def prepare_lm_inputs_labels(text):
    """
    Shift word sequences by 1 position so that the target for position (i) is
    word at position (i+1). The model will use all words up till position (i)
    to predict the next word.
    """
    text = tensorflow.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


text_ds = text_ds.map(prepare_lm_inputs_labels, num_parallel_calls=tf_data.AUTOTUNE)
text_ds = text_ds.prefetch(tf_data.AUTOTUNE)


50000 files


## Implement a Keras callback for generating text

In [7]:

class TextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10, print_every=1
    ):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k

    def sample_from(self, logits):
        logits, indices = ops.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(ops.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:maxlen]
                sample_index = maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x, verbose=0)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")


# Tokenize starting prompt
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index

start_prompt = "this movie is"
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]
num_tokens_generated = 40
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, vocab)


## Train the model

Note: This code should preferably be run on GPU.

In [8]:
model = create_model()

#Calculating carbon emissions for tracking

In [ ]:
!pip install codecarbon
from codecarbon import EmissionsTracker

In [10]:
#start emissions tracking
tracker = EmissionsTracker()
tracker.start()

model.fit(text_ds, verbose=2, epochs=25, callbacks=[text_gen_callback])

#stop emissions tracking
emissions: float = tracker.stop()
print(f"Emissions during training: {emissions} kg CO2")

[codecarbon WARNING @ 18:09:18] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:09:18] [setup] RAM Tracking...
[codecarbon INFO @ 18:09:18] [setup] CPU Tracking...
[codecarbon WARNING @ 18:09:19] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:09:19] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:09:19] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 18:09:19] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:09:19] [setup] GPU Tracking...
[codecarbon INFO @ 18:09:19] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:09:19] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

Epoch 1/25


[codecarbon INFO @ 18:09:34] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:09:34] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:09:34] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 18:09:34] Energy consumed for all GPUs : 0.000180 kWh. Total GPU Power : 43.03518079101798 W
[codecarbon INFO @ 18:09:34] 0.000398 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:09:49] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:09:49] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:09:49] Energy consumed for All CPU : 0.000354 kWh
[codecarbon INFO @ 18:09:49] Energy consumed for all GPUs : 0.000440 kWh. Total GPU Power : 62.443626008941315 W
[codecarbon INFO @ 18:09:49] 0.000878 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:10:04] Energy consumed for RAM : 0.000125 kWh. RAM Power : 

generated text:
this movie is one of the best of my favourite . i have never seen this , but this film would have no longer have been done with the most underrated . the first time of the first , i 'm not one of

391/391 - 82s - 210ms/step - loss: 5.4551
Epoch 2/25


[codecarbon INFO @ 18:10:49] Energy consumed for RAM : 0.000250 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:10:49] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:10:49] Energy consumed for All CPU : 0.001063 kWh
[codecarbon INFO @ 18:10:49] Energy consumed for all GPUs : 0.001509 kWh. Total GPU Power : 52.14966673272182 W
[codecarbon INFO @ 18:10:49] 0.002821 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:04] Energy consumed for RAM : 0.000292 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:11:04] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:11:04] Energy consumed for All CPU : 0.001239 kWh
[codecarbon INFO @ 18:11:04] Energy consumed for all GPUs : 0.001798 kWh. Total GPU Power : 69.48058215820235 W
[codecarbon INFO @ 18:11:04] 0.003329 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:19] Energy consumed for RAM : 0.000333 kWh. RAM Power : 1

...

generated text:
this movie is one of the funniest movies i have seen in my life . it makes you cringe at the time . i have a great idea to have the potential to be a bad idea to make movies about the characters ,

391/391 - 82s - 210ms/step - loss: 3.3418
Epoch 25/25


[codecarbon INFO @ 18:40:20] Energy consumed for RAM : 0.005164 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:40:20] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:40:20] Energy consumed for All CPU : 0.021948 kWh
[codecarbon INFO @ 18:40:20] Energy consumed for all GPUs : 0.031310 kWh. Total GPU Power : 32.098578135609095 W
[codecarbon INFO @ 18:40:20] 0.058422 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:40:35] Energy consumed for RAM : 0.005205 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:40:35] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:40:35] Energy consumed for All CPU : 0.022125 kWh
[codecarbon INFO @ 18:40:35] Energy consumed for all GPUs : 0.031555 kWh. Total GPU Power : 58.9948365685441 W
[codecarbon INFO @ 18:40:35] 0.058886 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:40:50] Energy consumed for RAM : 0.005247 kWh. RAM Power : 1

generated text:
this movie is one of the best i have seen in the first place in the late 70s , it follows the footsteps of the likes of chris chan films and john cusack are having the other two films . i am sorely disappointed

391/391 - 82s - 210ms/step - loss: 3.3236


[codecarbon INFO @ 18:41:35] Energy consumed for RAM : 0.005372 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:41:35] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 18:41:35] Energy consumed for All CPU : 0.022833 kWh
[codecarbon INFO @ 18:41:35] Energy consumed for all GPUs : 0.032613 kWh. Total GPU Power : 44.72857464741342 W
[codecarbon INFO @ 18:41:35] 0.060818 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:41:46] Energy consumed for RAM : 0.005403 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:41:46] Delta energy consumed for CPU with constant : 0.000130 kWh, power : 42.5 W
[codecarbon INFO @ 18:41:46] Energy consumed for All CPU : 0.022963 kWh
[codecarbon INFO @ 18:41:46] Energy consumed for all GPUs : 0.032710 kWh. Total GPU Power : 31.849788595112145 W
[codecarbon INFO @ 18:41:46] 0.061076 kWh of electricity used since the beginning.


Emissions during training: 0.008474259048095772 kg CO2


In [11]:
emissions_data = tracker.final_emissions_data
print(f"Energy consumed: {emissions_data.energy_consumed:.6f} kWh")
print(f"Execution Time: {emissions_data.duration:.2f} seconds")
print(f"Emissions: {emissions_data.emissions} kg CO2")

Energy consumed: 0.061076 kWh
Execution Time: 1946.22 seconds
Emissions: 0.008474259048095772 kg CO2
